In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import keras_tuner as kt

2025-04-23 23:06:38.886716: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
CSV_PATH      = "../ml/data/processed/china_mill_data_2025_03_04_09_30_30.csv"
COL_NAME      = "energy"
SPLIT_RATIO   = 0.8
SHUFFLE_BUF   = 1_000

# You can tune these, or leave defaults for Keras-Tuner to override:
DEFAULT_WINDOW = 60
DEFAULT_BATCH  = 32
EPOCHS         = 50


In [3]:
df = pd.read_csv(CSV_PATH)
series = df[COL_NAME].astype(np.float32).values

split_idx   = int(len(series) * SPLIT_RATIO)
train_series = series[:split_idx]
val_series   = series[split_idx:]

In [6]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1])) 
    return ds.batch(batch_size).prefetch(1)

In [7]:
train_ds = windowed_dataset(train_series, DEFAULT_WINDOW, DEFAULT_BATCH, SHUFFLE_BUF)
val_ds   = windowed_dataset(val_series,   DEFAULT_WINDOW, DEFAULT_BATCH, SHUFFLE_BUF)

2025-04-23 23:09:05.003878: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-23 23:09:05.004153: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
def build_model(window_size, 
                conv_filters=64, 
                lstm_units=64, 
                dropout_rate=0.2, 
                learning_rate=1e-3):
    x   = tf.keras.Input(shape=(window_size,1))
    x   = tf.keras.layers.Conv1D(conv_filters, 5, padding="causal", activation="relu")(x)
    x   = tf.keras.layers.LSTM(lstm_units, return_sequences=True)(x)
    x   = tf.keras.layers.Dropout(dropout_rate)(x)
    x   = tf.keras.layers.LSTM(lstm_units)(x)
    x   = tf.keras.layers.Dropout(dropout_rate)(x)
    out = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inp, out, name="energy_forecaster")

    model.compile(
        loss=tf.keras.losses.Huber(),
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        metrics=["mae"]
    )
    return model

In [13]:
model = build_model(DEFAULT_WINDOW)

ModuleNotFoundError: No module named 'keras.api._v2'

In [ ]:
model.summary()

In [ ]:
callbacks = [
    # 1) Stop early if no improvement on val_mae
    tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=10, restore_best_weights=True
    ),
    # 2) Reduce LR when plateau
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_mae", factor=0.5, patience=5, min_lr=1e-6
    ),
    # 3) Save checkpoint every epoch
    tf.keras.callbacks.ModelCheckpoint(
        filepath="models/epoch_{epoch:02d}.h5",
        save_best_only=False
    ),
    # 4) TensorBoard
    tf.keras.callbacks.TensorBoard(log_dir="logs/energy")
]

In [ ]:
def build_tuner_model(hp):
    ws = hp.Int("window_size", 30, 3360, step=30)         # can tune from 0.5h up to a week
    cf = hp.Choice("conv_filters", [32, 64, 128])
    lu = hp.Choice("lstm_units",   [32, 64, 128])
    dr = hp.Float("dropout", 0.0, 0.5, step=0.1)
    lr = hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")

    # rebuild datasets with this window size
    train_ds_t = windowed_dataset(train_series, ws, DEFAULT_BATCH, SHUFFLE_BUF)
    val_ds_t   = windowed_dataset(val_series,   ws, DEFAULT_BATCH, SHUFFLE_BUF)

    m = build_model(ws, conv_filters=cf, lstm_units=lu, dropout_rate=dr, learning_rate=lr)
    m.fit(
        train_ds_t,
        validation_data=val_ds_t,
        epochs=20,         # short for tuning
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=5, restore_best_weights=True)
        ],
        verbose=0
    )
    return m


In [ ]:
ner = kt.Hyperband(
    build_tuner_model,
    objective="val_mae",
    max_epochs=50,
    factor=3,
    directory="ktuner",
    project_name="energy_forecast"
)

In [ ]:
best_hp    = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.get_best_models(1)[0]

In [ ]:
print("Best window_size:",     best_hp.get("window_size"))
print("Best conv_filters:",    best_hp.get("conv_filters"))
print("Best lstm_units:",      best_hp.get("lstm_units"))
print("Best dropout_rate:",    best_hp.get("dropout"))
print("Best learning_rate:",   best_hp.get("learning_rate"))

In [ ]:
ws       = best_hp.get("window_size")
train_ds = windowed_dataset(train_series, ws, DEFAULT_BATCH, SHUFFLE_BUF)
val_ds   = windowed_dataset(val_series,   ws, DEFAULT_BATCH, SHUFFLE_BUF)

In [ ]:
history = best_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(history.history["loss"],      label="train loss")
plt.plot(history.history["val_loss"],  label="val loss")
plt.plot(history.history["mae"],       label="train mae")
plt.plot(history.history["val_mae"],   label="val mae")
plt.legend(); plt.grid(True); plt.title("History")

In [ ]:
for x_b, y_b in val_ds.take(1):
    preds = best_model.predict(x_b)
    hist  = x_b[0,:,0].numpy()
    true  = y_b[0].numpy()
    pred  = preds[0,0]
    plt.figure(figsize=(12,4))
    plt.plot(range(ws), hist, label="history")
    plt.scatter(ws, true, c="green", label="true next")
    plt.scatter(ws, pred, c="red",   label="predicted next")
    plt.legend(); plt.grid(True); plt.title("Demo Prediction")
    plt.show()
    break